<a href="https://colab.research.google.com/github/K-Erath/Dataquest/blob/master/13_Guided_Project_Building_a_Spam_Filter_with_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Spam Filter with Naive Bayes
In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Exploring the Dataset
We'll now start by reading in the dataset.

In [1]:
import urllib.request 
import pandas as pd

In [2]:
url = "https://dq-content.s3.amazonaws.com/433/SMSSpamCollection"
fname = "SMSSpamCollection"
df = pd.read_csv(fname, sep='\t', header=None, names=['Label', 'SMS'])
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.shape

(5572, 2)

In [4]:
df["Label"].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

Below, we see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

In [5]:
df["Label"].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Training and Test Set
We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [6]:
df_randomized = df.sample(frac=1, random_state=1).reset_index(drop=True)

In [7]:
df_train = df_randomized.iloc[1114:, :].copy().reset_index(drop=True)
df_test = df_randomized.iloc[:1114, :].copy().reset_index(drop=True)

In [8]:
df_train.shape

(4458, 2)

In [9]:
df_test.shape

(1114, 2)


We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [10]:
df_test["Label"].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

In [11]:
df_train["Label"].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64


The results look good. We'll now move on to cleaning the dataset.

## Data Cleaning
To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

Essentially, we want to bring data to this format:
![format](https://drive.google.com/uc?id=13qIrngsreHScgRRyE3ElOsZX0Ka9CKV2)

### Letter Case and Punctuation
We'll begin with removing all the punctuation and bringing every letter to lower case.

In [12]:
# remove punctuation
df_train["SMS"] = df_train["SMS"].str.replace('\W', ' ')
# convert to all lower case
df_train["SMS"] = df_train["SMS"].str.lower()

df_train.head()

,Label,SMS
0,ham,yeah do don t stand to close tho you ll catc...
1,ham,hi where are you we re at and they re not ...
2,ham,if you r home then come down within 5 min
3,ham,when re you guys getting back g said you were...
4,ham,tell my bad character which u dnt lik in me ...


### Creating the Vocabulary
Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [13]:
df_train['SMS'] = df_train['SMS'].str.split()

vocabulary = []
for row in df_train["SMS"]:
  for word in row:
    vocabulary.append(word)

vocabulary = list(set(vocabulary))

It looks like there are 7,783 unique words in all the messages of our training set.

In [14]:
len(vocabulary)

7753

### The Final Training Set
We're now going to use the vocabulary we just created to make the data transformation we want.

In [15]:
word_counts_per_sms = {word: [0] * len(df_train['SMS']) for word in vocabulary}

for index, sms in enumerate(df_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [16]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,scratching,dined,famamus,espe,87239,hv,call2optout,perform,gona,jjc,borrow,reliant,enemy,tough,shampain,comfort,wkent,greet,r836,servs,dialogue,bang,laxinorficated,marine,breaking,ur,landmark,sufficient,saristar,iraq,dlf,banks,dwn,ü,tenerife,feel,wheat,postcode,viva,toking,...,32,8wp,craving,payee,forward,suckers,ball,general,parked,sw7,customercare,write,had,erupt,evening,zhong,tablet,ish,edison,fortune,dough,tootsie,cheer,brain,01223585236,via,babysitting,hello,subject,luxury,dirt,styles,fresh,points,least5times,loss,george,09064019014,yourinclusive,mah
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
df_train = pd.concat([df_train, word_counts], axis=1)
df_train.head()

,Label,SMS,scratching,dined,famamus,espe,87239,hv,call2optout,perform,gona,jjc,borrow,reliant,enemy,tough,shampain,comfort,wkent,greet,r836,servs,dialogue,bang,laxinorficated,marine,breaking,ur,landmark,sufficient,saristar,iraq,dlf,banks,dwn,ü,tenerife,feel,wheat,postcode,...,32,8wp,craving,payee,forward,suckers,ball,general,parked,sw7,customercare,write,had,erupt,evening,zhong,tablet,ish,edison,fortune,dough,tootsie,cheer,brain,01223585236,via,babysitting,hello,subject,luxury,dirt,styles,fresh,points,least5times,loss,george,09064019014,yourinclusive,mah
0,ham,"[yeah, do, don, t, stand, to, close, tho, you,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ham,"[hi, where, are, you, we, re, at, and, they, r...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ham,"[if, you, r, home, then, come, down, within, 5...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ham,"[when, re, you, guys, getting, back, g, said, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ham,"[tell, my, bad, character, which, u, dnt, lik,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First
We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set $\alpha = 1$.

In [18]:
# isolate spam messages
df_spam = df_train[df_train['Label'] == 'spam']
df_spam.head()

,Label,SMS,scratching,dined,famamus,espe,87239,hv,call2optout,perform,gona,jjc,borrow,reliant,enemy,tough,shampain,comfort,wkent,greet,r836,servs,dialogue,bang,laxinorficated,marine,breaking,ur,landmark,sufficient,saristar,iraq,dlf,banks,dwn,ü,tenerife,feel,wheat,postcode,...,32,8wp,craving,payee,forward,suckers,ball,general,parked,sw7,customercare,write,had,erupt,evening,zhong,tablet,ish,edison,fortune,dough,tootsie,cheer,brain,01223585236,via,babysitting,hello,subject,luxury,dirt,styles,fresh,points,least5times,loss,george,09064019014,yourinclusive,mah
8,spam,"[camera, you, are, awarded, a, sipix, digital,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,spam,"[500, free, text, msgs, just, text, ok, to, 80...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,spam,"[you, have, won, as, a, valued, vodafone, cust...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31,spam,"[auction, round, 4, the, highest, bid, is, now...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32,spam,"[bored, housewives, chat, n, date, now, 087175...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# isolate ham messages
df_ham = df_train[df_train['Label'] == 'ham']
df_ham.head()

,Label,SMS,scratching,dined,famamus,espe,87239,hv,call2optout,perform,gona,jjc,borrow,reliant,enemy,tough,shampain,comfort,wkent,greet,r836,servs,dialogue,bang,laxinorficated,marine,breaking,ur,landmark,sufficient,saristar,iraq,dlf,banks,dwn,ü,tenerife,feel,wheat,postcode,...,32,8wp,craving,payee,forward,suckers,ball,general,parked,sw7,customercare,write,had,erupt,evening,zhong,tablet,ish,edison,fortune,dough,tootsie,cheer,brain,01223585236,via,babysitting,hello,subject,luxury,dirt,styles,fresh,points,least5times,loss,george,09064019014,yourinclusive,mah
0,ham,"[yeah, do, don, t, stand, to, close, tho, you,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ham,"[hi, where, are, you, we, re, at, and, they, r...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ham,"[if, you, r, home, then, come, down, within, 5...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ham,"[when, re, you, guys, getting, back, g, said, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ham,"[tell, my, bad, character, which, u, dnt, lik,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# calculate probability of spam
p_spam = len(df_spam) / len(df_train)
p_spam

0.13458950201884254

In [21]:
# calculate probability of ham
p_ham = len(df_ham) / len(df_train)
p_ham

0.8654104979811574

In [22]:
# total number of spam words
n_spam = df_spam.drop(['Label', 'SMS'], axis=1).sum().sum()
n_spam

15275

In [23]:
# total number of ham words
n_ham = df_ham.drop(['Label', 'SMS'], axis=1).sum().sum()
n_ham

56649

In [24]:
# number of words in vocabulary (total unique words, spam and ham)
n_vocab = len(vocabulary)
n_vocab

7753

In [25]:
# laplace smoothing
alpha = 1

## Calculating Parameters
Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [26]:
# initiate parameters
p_spam_dict = {word: 0 for word in vocabulary}
p_ham_dict = {word: 0 for word in vocabulary}

# calculate parameters
for word in vocabulary:
    n_word = df_spam[word].sum()
    p = (n_word + alpha) / (n_spam + alpha*n_vocab)
    p_spam_dict[word] = p

    n_word = df_ham[word].sum()
    p = (n_word + alpha) / (n_ham + alpha*n_vocab)
    p_ham_dict[word] = p

## Classifying A New Message
Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w1, w2, ..., wn).
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    * If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    * If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [27]:
import re

In [28]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
  
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in vocabulary:
            p_spam_given_message *= p_spam_dict[word]
            p_ham_given_message *= p_ham_dict[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [29]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.2784957584472927e-25
P(Ham|message): 2.5841428475044265e-27
Label: Spam


In [30]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.774748444294843e-25
P(Ham|message): 3.455584370145657e-21
Label: Ham


## Measuring the Spam Filter's Accuracy
The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [31]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
  
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in vocabulary:
            p_spam_given_message *= p_spam_dict[word]
            p_ham_given_message *= p_ham_dict[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'need human'


Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [32]:
df_test['predicted'] = df_test['SMS'].apply(classify_test_set)
df_test.head()

,Label,SMS,predicted
0,ham,"Yep, by the pretty sculpture",ham
1,ham,"Yes, princess. Are you going to make me moan?",ham
2,ham,Welp apparently he retired,ham
3,ham,Havent.,ham
4,ham,I forgot 2 ask ü all smth.. There's a card on ...,ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [33]:
total = df_test.shape[0]
correct = df_test[df_test['Label'] == df_test['predicted']].shape[0]

print('Correct:', correct)
print('Incorrect:', total - correct)
print(f'Accuracy: {round(correct/total * 100, 2)}%')

Correct: 1101
Incorrect: 13
Accuracy: 98.83%


The accuracy is close to 98.83%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,101 correctly.

## Summary
In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.83% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.